In [24]:
from __future__ import annotations

import ibis
from ibis import _
from ibis.expr.types import (
    BooleanValue,
    Table,
    IntegerValue,
    StringValue,
    FloatingValue,
    StructValue,
)

from mismo.datasets import load_patents

ibis.options.interactive = True

We are going to dedupe the PATSTAT dataset. Every record represents a patent filed,
and our task is to determine which one came from the same inventor.
The end goal is to add a column called `inventor_id` to each patent.
This dataset contains a ground truth label, so we can evaluate how well we did.

`Mismo` contains this as an included dataset so it is easy to get started.
The returned dataset is an Ibis table, which is a lazy representation of a SQL table.
It is similar to a pandas dataframe, but has a few properties that make it much
better for the record linkage use case:

- Since it is SQL backed, it can handle datasets that are larger than memory, in
  the many millions of rows.
- Computation is performed by the powerful SQL backend of your choice: Google BigQuery,
  Apache Spark, Snowflake, etc. For this demo, we use DuckDB, which is a 
  state-of-the-art SQL engine based around a columnar data model
  (ie oriented towards the bulk operations of record linkage)
- Ibis is strongly typed, has a full API, is well-documented, and has good
  integration with the rest of the python data science ecosystem.


In [2]:
patents = load_patents()
print(patents.count())
patents

2379



┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ latitude ┃ longitude ┃ coauthors                                                                        ┃ classes                                                          ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ float64  │ float64   │ string                                                                           │ string                                                           │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────┼───────────┼──────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │     0.00 │  0.000000 │ KONINK PHILIPS ELECTRONICS N V**DAVID E  SNYDER**THOMAS D  LYSTER                │ A61N**A61B                                                       │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │     0.00 │  0.000000 │ TSJERK  HOEKSTRA**ANDRESS K  JOHNSON**TERESA MARIE  CHERON**ALBERTO  SLIKTA**JA… │ G01N**B01L**C11D**G02F**F16L                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │     0.00 │  0.000000 │ WILLIAM JOHN ERNEST  PARR**HANS  OSKARSSON**MARTIN  HELLSTEN**KORNELIS  OVERKEM… │ C09K**F17D**B01F**C23F                                           │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ GUENTER  KOCHSMEIER**ZBIGNIEW  WIEGOLASKI**EVAN JOHN  STANBURY**PETER GRANT  JE… │ G02B**G04G**H02G**G06F                                           │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │    52.35 │  4.916667 │ ZILAN  MANFRED**JOSIANE  RAMOS**DUANE LYNN  MORTENSEN**CHRISTIAN  LE SERGENT     │ H03G**B05D**H04L**H04B**C03B**C03C**G02B**H01B                   │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │     0.00 │  0.000000 │ OLIVIER  AUDOUIN**MICHEL  SOTOM**JEAN MICHEL  GABRIAGUES                         │ H04B**H01S**H04J                                                 │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │     0.00 │  0.000000 │ LEE  RICKLER**SIMON  PARKER**CANON RES CENT EURO **RAKEFET  SAGMAN**TIMOTHY FRA… │ G06F                                                             │
│     15042 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V.          │     0.00 │  0.000000 │ QI HE  HONG**ADAM MICHAEL  BAUMBERG**ALEXANDER RALPH  LYONS                      │ G06T**G01B                                                       │
│     15043 │    4333661 │ CANON EUROPA         │ * CANON EUROPA NV            │     0.00 │  0.000000 │ NILESH  PATHAK**MASAMICHI  MASUDA** CANON TECHNOLOGY EURO **PATRICK WILLIAM  MO… │ H04B**G06T**G06F**H04M**H04N**H04Q**G03B**B41J**G01B**G06Q       │
│     25387 │    7650783 │ DSM                  │ * DSM N.V.                   │     0.00 │  0.000000 │ GABRIEL MARINUS  MEESTERS**RUDOLF CAROLUS  BARENDSE**ARIE KARST  KIES**ALEXANDE… │ C12N**A61K**A23L**A23J**A23K**A01H**B01J**C12R**C07D**A61P**B01D │
│         … │          … │ …                    │ …                            │        … │         … │ …                                                                                │ …  

Let's clean this up a bit:
- clean up whitespace
- convert the `coauthors` and `classes` columns to actual arrays (they really represent sets)

Each element in `classes` is a 4-character IPC technical code that is like a tag
for the patent. Similar patents will have similar tags.


In [4]:
from mismo.clean.strings import norm_whitespace  # noqa: E402


def clean_names(names: StringValue) -> StringValue:
    names = norm_whitespace(names)
    names = names.upper()
    # Only want to keep letters, numbers, and spaces
    names = names.re_replace("[^0-9A-Z ]", "")
    # Now have to do whitespace fixup again
    names = norm_whitespace(names)
    return names


cleaned = patents.select(
    "record_id",
    "label_true",
    "name_true",
    "name",
    name_cleaned=clean_names(_.name),
    latitude=_.latitude.nullif(0),
    longitude=_.longitude.nullif(0),
    coauthors=_.coauthors.upper().split("**").map(norm_whitespace).sort(),
    classes=_.classes.upper().split("**").map(norm_whitespace).sort(),
)

Then, let's add some features. The binned coordinates will be used in the blocking step,
so that locations in the same lat/lng bin will be compared to each other.

We also generate some features based on the `name` column.

In [34]:
def bin_lat_lon(lat: FloatingValue, lon: FloatingValue) -> StructValue:
    """Bin a latitude or longitude to 0.1 degree precision, which is ~6 miles.

    If both are null, return null.

    (52.35, 4.916667 -> (524, 49)
    """

    def _bin_coord(coord: FloatingValue) -> IntegerValue:
        return (coord.round(1) * 10).cast("int16").fillna(0)

    result = ibis.struct(
        {
            "lat_hash": _bin_coord(lat),
            "lon_hash": _bin_coord(lon),
        }
    )
    both_null = lat.isnull() & lon.isnull()
    return both_null.ifelse(ibis.NA, result)


featured = cleaned.mutate(
    name_tokens=_.name_cleaned.split(" ").map(norm_whitespace).sort(),
    name_first3=_.name_cleaned[0:3],
    coords_hashed=bin_lat_lon(_.latitude, _.longitude),
)
featured

┏━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ record_id ┃ label_true ┃ name_true            ┃ name                         ┃ name_cleaned             ┃ latitude ┃ longitude ┃ coauthors                                                                           ┃ classes                  ┃ name_tokens                 ┃ name_first3 ┃ coords_hashed                            ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ int64     │ int64      │ string               │ string                       │ string                   │ float64  │ float64   │ array<string>                                                                       │ array<string>            │ array<string>               │ string      │ struct<lat_hash: int16, lon_hash: int16> │
├───────────┼────────────┼──────────────────────┼──────────────────────────────┼──────────────────────────┼──────────┼───────────┼─────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────┼─────────────────────────────┼─────────────┼──────────────────────────────────────────┤
│      2909 │     402600 │ AGILENT TECHNOLOGIES │ * AGILENT TECHNOLOGIES, INC. │ AGILENT TECHNOLOGIES INC │      nan │       nan │ ['DAVID E SNYDER', 'KONINK PHILIPS ELECTRONICS N V', ... +1]                        │ ['A61B', 'A61N']         │ ['AGILENT', 'INC', ... +1]  │ AGI         │ NULL                                     │
│      3574 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL N.V.            │ AKZO NOBEL NV            │      nan │       nan │ ['ALBERT BORREMAN', 'ALBERTO SLIKTA', ... +11]                                      │ ['B01L', 'C11D', ... +3] │ ['AKZO', 'NOBEL', ... +1]   │ AKZ         │ NULL                                     │
│      3575 │     569309 │ AKZO NOBEL           │ * AKZO NOBEL NV              │ AKZO NOBEL NV            │      nan │       nan │ ['HANS OSKARSSON', 'JOHANNA CHRISTINA SPEELMAN', ... +3]                            │ ['B01F', 'C09K', ... +2] │ ['AKZO', 'NOBEL', ... +1]   │ AKZ         │ NULL                                     │
│      3779 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │    52.35 │  4.916667 │ ['EVAN JOHN STANBURY', 'GUENTER KOCHSMEIER', ... +2]                                │ ['G02B', 'G04G', ... +2] │ ['ALCATEL', 'NV']           │ ALC         │ {'lat_hash': 524, 'lon_hash': 49}        │
│      3780 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │    52.35 │  4.916667 │ ['CHRISTIAN LE SERGENT', 'DUANE LYNN MORTENSEN', ... +2]                            │ ['B05D', 'C03B', ... +6] │ ['ALCATEL', 'NV']           │ ALC         │ {'lat_hash': 524, 'lon_hash': 49}        │
│      3782 │     656303 │ ALCATEL              │ * ALCATEL N.V.               │ ALCATEL NV               │      nan │       nan │ ['JEAN MICHEL GABRIAGUES', 'MICHEL SOTOM', ... +1]                                  │ ['H01S', 'H04B', ... +1] │ ['ALCATEL', 'NV']           │ ALC         │ NULL                                     │
│     15041 │    4333661 │ CANON EUROPA         │ * CANON EUROPA N.V           │ CANON EUROPA NV          │      nan │       nan │ ['CANON RES CENT EURO', 'LEE RICKLER', ... +3]                                      │ ['G06F']                 │ ['CANON', 'EUROPA', ... +1] │ CAN         │ NULL                                     │
│     15042 │    4333661 │ C

OK, now it's time to block! This is where we generate comparisons between records.
If we were naive and generated all possible comparisons from N record,
you would end up with N^2 comparisons. For our small dataset of ~2000 records
we would be able to get away with this, but for datasets much larger than this
it would be infeasible.

In [36]:
from mismo.block import BlockingRule, BlockingRules


rules = BlockingRules(
    BlockingRule(
        "coords_close", lambda left, right: left.coords_hashed == right.coords_hashed
    ),
    BlockingRule(
        "names_exact", lambda left, right: left.name_cleaned == right.name_cleaned
    ),
)

featured = featured.cache()
blocked = rules.block(featured, featured)
blocked

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                 ┃ classes_r                ┃ coauthors_l                                                                 ┃ coauthors_r                                                                        ┃ coords_hashed_l                          ┃ coords_hashed_r                          ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                                ┃ name_cleaned_r                                ┃ name_first3_l ┃ name_first3_r ┃ name_l                                            ┃ name_r                                           ┃ name_tokens_l                         ┃ name_tokens_r                         ┃ name_true_l               ┃ name_true_r               ┃ blocking_rule ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ int64       │ int64       │ array<string>             │ array<string>            │ array<string>                                                               │ array<string>                                                                      │ struct<lat_hash: int16, lon_hash: int16> │ struct<lat_hash: int16, lon_hash: int16> │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string                                        │ string                                        │ string        │ string        │ string                                            │ string                                           │ array<string>                         │ array<string>                         │ string                    │ string                    │ string        │
├─────────────┼─────────────┼───────────────────────────┼──────────────────────────┼─────────────────────────────────────────────────────────────────────────────┼────────────────────────────────────────────────────────────────────────────────────┼──────────────────────────────────────────┼──────────────────────────────────────────┼──────────────┼──────────────┼────────────┼────────────┼─────────────┼─────────────┼───────────────────────────────────────────────┼───────────────────────────────────────────────┼───────────────┼───────────────┼───────────────────────────────────────────────────┼──────────────────────────────────────────────────┼───────────────────────────────────────┼───────────────────────────────────────┼───────────────────────────┼───────────────────────────┼────────────

The result of that was the two tables joined together, with a `_l` added
to all the columns from the left table, and a `_r` added to all the columns
from the right table.

By blocking, we reduced the number of needed comparisons by a large factor.
In larger datasets, and with better blocking rules, this would be even more!


In [37]:
from mismo import metrics

n_comparisons = blocked.count().execute()
n_naive = metrics.n_naive_comparisons(featured)
reduction_ratio = n_comparisons / n_naive
n_naive, n_comparisons, reduction_ratio

(2828631, 450905, 0.1594075013672692)

OK, now that we have our candidate comparisons generated, let's actually do the
comparing of pairs. There are many ways to do this, but one of the most common
is to generate a set of features.

In this case, we will generate a features for
- how well the names match
    - exact match
    - the first three letters match
    - everything else (this is implicit, we don't actually pass this feature)
- how well the set of classes match
    - exact match
    - the [jaccard index](https://en.wikipedia.org/wiki/Jaccard_index) of the sets of classes is above 50%
    - everything else

In [43]:
from mismo.compare import (  # noqa: E402
    Comparison,
    ComparisonLevel,
    exact_level,
    jaccard,
    distance_km,
)
from mismo.compare.fs import FellegiSunterComparer  # noqa: E402

name_comparison = Comparison(
    name="Name",
    levels=[
        exact_level("name_cleaned"),
        exact_level("name_first3"),
    ],
)

classes_comparison = Comparison(
    name="Classes",
    levels=[
        exact_level("classes"),
        ComparisonLevel(
            name="Classes 50% Jaccard",
            condition=lambda table: jaccard(table.classes_l, table.classes_r) >= 0.5,
        ),
    ],
)

coauthors_comparison = Comparison(
    name="Coauthors",
    levels=[
        exact_level("coauthors"),
        ComparisonLevel(
            name="Coauthors 50% Jaccard",
            condition=lambda table: jaccard(table.coauthors_l, table.coauthors_r)
            >= 0.5,
        ),
    ],
)

coords_comparison = Comparison(
    name="Coords",
    levels=[
        ComparisonLevel(
            name="Coords match",
            condition=lambda table: (table.latitude_l == table.latitude_r)
            & (table.longitude_l == table.longitude_r),
        ),
        ComparisonLevel(
            name="Coords within 100km",
            condition=lambda table: distance_km(
                lat1=table.latitude_l,
                lon1=table.longitude_l,
                lat2=table.latitude_r,
                lon2=table.longitude_r,
            )
            <= 100,
        ),
        # TODO: deal with if one of the coordinates is missing
    ],
)

comparisons = [
    name_comparison,
    classes_comparison,
    coauthors_comparison,
    coords_comparison,
]
fs = FellegiSunterComparer(comparisons)
fs_trained = fs.trained(featured, featured, max_pairs=10_000, seed=42)
# Let's guess that every record has 3 matches. This isn't that important,
# it just makes the probabilities more reasonable, but regardless of the
# absolute value of the predicted probabilities, the relative values are
# what matter.
fs_trained.weights.prior = 3 / featured.count().execute()
fs_trained

FellegiSunterComparer(Comparisons(Comparison(name=Name, levels=[ComparisonLevel(name=exact_name_cleaned, description=Exact match on `name_cleaned`), ComparisonLevel(name=exact_name_first3, description=Exact match on `name_first3`)]), Comparison(name=Classes, levels=[ComparisonLevel(name=exact_classes, description=Exact match on `classes`), ComparisonLevel(name=Classes 50% Jaccard)]), Comparison(name=Coauthors, levels=[ComparisonLevel(name=exact_coauthors, description=Exact match on `coauthors`), ComparisonLevel(name=Coauthors 50% Jaccard)]), Comparison(name=Coords, levels=[ComparisonLevel(name=Coords match), ComparisonLevel(name=Coords within 100km)])), <mismo.compare.fs._base.Weights object at 0x13a440790>)

In [44]:
compared = fs_trained.compare(blocked)
compared = compared.cache()
compared

┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━━━┓
┃ record_id_l ┃ record_id_r ┃ classes_l                 ┃ classes_r                ┃ coauthors_l                                                                                                               ┃ coauthors_r                                                                                                               ┃ coords_hashed_l                          ┃ coords_hashed_r                          ┃ label_true_l ┃ label_true_r ┃ latitude_l ┃ latitude_r ┃ longitude_l ┃ longitude_r ┃ name_cleaned_l                    ┃ name_cleaned_r                    ┃ name_first3_l ┃ name_first3_r ┃ name_l                              ┃ name_r                                          ┃ name_tokens_l               ┃ name_tokens_r               ┃ name_true_l                ┃ name_true_r                ┃ blocking_rule ┃ Name_cmp           ┃ Name_bf  ┃ Classes_cmp   ┃ Classes_bf ┃ Coauthors_cmp   ┃ Coauthors_bf ┃ Coords_cmp          ┃ Coords_bf ┃ bf       ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━━━┩
│ int64       │ int64       │ array<string>             │ array<string>            │ array<string>                                                                                                             │ array<string>                                                                                                             │ struct<lat_hash: int16, lon_hash: int16> │ struct<lat_hash: int16, lon_hash: int16> │ int64        │ int64        │ float64    │ float64    │ float64     │ float64     │ string                            │ string                            │ string        │ string        │ string                              │ string                                          │ array<string>               │ array<string>               │ string                     │ string                     │ string        │ string             │ float64  │ string        │ float64    │ string          │ float64      │ string              │ float64   │ float64  │
├─────────────┼─────────────┼───────────────────────────┼──────────────────────────┼────────────────────────────────────────────

Look at the different combinations of comparisons that we found, and see which ones
led to the best match, and which ones to the worst.

Unsurprisingly, the exact match levels have the highest bayes factor, and the
ELSE levels have the lowest. The other levels are somewhere in between.

In [46]:
comparison_columns = [comp.name + "_cmp" for comp in comparisons]
vector_counts = compared.group_by(comparison_columns).agg(
    n_pairs=_.count(),
    bf=_.bf.arbitrary(),
)
vector_counts.order_by(_.bf.desc()).head(5), vector_counts.order_by(_.bf.asc()).head(5)

(┏━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━┓
 ┃ Name_cmp           ┃ Classes_cmp         ┃ Coauthors_cmp         ┃ Coords_cmp   ┃ n_pairs ┃ bf       ┃
 ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━┩
 │ string             │ string              │ string                │ string       │ int64   │ float64  │
 ├────────────────────┼─────────────────────┼───────────────────────┼──────────────┼─────────┼──────────┤
 │ exact_name_cleaned │ exact_classes       │ exact_coauthors       │ Coords match │    2992 │ 1.287064 │
 │ exact_name_first3  │ exact_classes       │ exact_coauthors       │ Coords match │       6 │ 1.120667 │
 │ exact_name_cleaned │ Classes 50% Jaccard │ exact_coauthors       │ Coords match │       8 │ 0.888589 │
 │ exact_name_cleaned │ exact_classes       │ Coauthors 50% Jaccard │ Coords match │      12 │ 0.304831 │
 │ exact_name_cleaned │ exact_classes       │ 

Let's plot all of these to try to find a threshold for what to count as a match

In [47]:
import altair as alt

alt.Chart(vector_counts.execute()).mark_bar().encode(
    x="bf",
    y=alt.Y(
        "n_pairs",
        scale=alt.Scale(type="log"),
    ),
    color="bf",
    tooltip=[
        "bf",
        "Name_cmp",
        "Classes_cmp",
        "Coauthors_cmp",
        "Coords_cmp",
    ],
)

alt.Chart(...)

It looks like a BF of .08 seems to separate the comparisons into two nice clusters.
If I hover over the above chart, I can see that pretty much all the ELSE comparisons
are in the low cluster, and all the SAME comparisons are in the high cluster.

In [48]:
bf_threshold = 0.08
(compared.bf >= bf_threshold).value_counts()

┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ GreaterEqual(bf, 0.08) ┃ GreaterEqual(bf, 0.08)_count ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ boolean                │ int64                        │
├────────────────────────┼──────────────────────────────┤
│ False                  │                       441263 │
│ True                   │                         9642 │
└────────────────────────┴──────────────────────────────┘

Let's be really picky and only take the most likely matches as true matches, and
then perform connected components to label each patent with its inventor:

In [49]:
from mismo.cluster import connected_components  # noqa: E402

links = compared[_.bf >= bf_threshold]
links = links.cache()
print(links.count())
labels = connected_components(links, nodes=featured.record_id)
labels

9642



┏━━━━━━━━━━━┳━━━━━━━━━━━┓
┃ record_id ┃ component ┃
┡━━━━━━━━━━━╇━━━━━━━━━━━┩
│ int64     │ int64     │
├───────────┼───────────┤
│  18320064 │  18320064 │
│  35244313 │  35244313 │
│  18321713 │  18321713 │
│  26827063 │  26827063 │
│  33977566 │  33977566 │
│  35244243 │  35244158 │
│  32091057 │  32091057 │
│  16362176 │  16362176 │
│     50633 │     50633 │
│   7651453 │   7651453 │
│         … │         … │
└───────────┴───────────┘

Now let's evaluate how good our labeling is. Mismo wraps all of the evaluation
metrics from sklearn, so we can use them with Ibis Tables.

In [50]:
labels_true = patents.select("record_id", label=_.label_true)
labels_pred = labels.select("record_id", label=_.component)
metrics.adjusted_rand_score(labels_true, labels_pred)

0.06762814686846828

In [51]:
metrics.homogeneity_score(labels_true, labels_pred)

0.9959583582542056

The high homogeneity means we have a high precision, and don't have a lot of false-links

In [52]:
metrics.completeness_score(labels_true, labels_pred)

0.4428130298345369

The low completeness score means we have low recall, and are missing a lot of true-links